<a href="https://colab.research.google.com/github/0xtunafresh/Pandas/blob/main/user_embedding_vestor_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:99: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(


In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cu116.html

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
df = pd.read_csv('/content/sample_data/kullanıcı_davranışı_verisi.csv')[:1000]

# LabelEncoder kullanarak kategorik sütunları encode et
le = LabelEncoder()
df['kategori_1'] = le.fit_transform(df['kategori_1'])
df['kategori_2'] = le.fit_transform(df['kategori_2'])
df['event_type'] = le.fit_transform(df['event_type'])

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/kullanıcı_davranışı_verisi.csv'

In [ ]:
# Prepare node features
node_features = df[['kategori_1', 'kategori_2']].values
node_features = torch.tensor(node_features, dtype=torch.float)


edge_index = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        if df['event_type'][i] == 1 and df['event_type'][j] == 1:
            edge_index.append([i, j])
            edge_index.append([j, i])
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()


NameError: name 'df' is not defined

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, num_features, embedding_dim):
        super().__init__()
        self.conv1 = GCNConv(num_features, embedding_dim)
        self.conv2 = GCNConv(embedding_dim, embedding_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x


In [ ]:
num_features = node_features.shape[1]
embedding_dim = 16  # Choose an appropriate embedding dimension
model = GNN(num_features, embedding_dim)

In [ ]:
model

GNN(
  (conv1): GCNConv(2, 16)
  (conv2): GCNConv(16, 16)
)

In [ ]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    embeddings = model(node_features, edge_index)

# Convert embeddings to a Pandas DataFrame for readability
embeddings_df = pd.DataFrame(embeddings.numpy())

# Display the embeddings
print(embeddings_df[:10])

         0         1         2         3         4         5         6   \
0 -0.563628 -0.493598  1.110309 -0.247862  1.343821 -2.239553 -1.819953   
1 -0.114103 -0.103755  0.053988 -0.350267  0.174056 -0.449888 -0.205385   
2 -0.114103 -0.103755  0.053988 -0.350267  0.174056 -0.449888 -0.205385   
3 -0.114103 -0.103755  0.053988 -0.350267  0.174056 -0.449888 -0.205385   
4 -0.114103 -0.103755  0.053988 -0.350267  0.174056 -0.449888 -0.205385   
5 -0.563628 -0.493598  1.110309 -0.247862  1.343821 -2.239553 -1.819953   
6 -0.553925 -0.496169  1.150250  0.024252  1.278846 -1.897882 -1.805178   
7 -0.553925 -0.496169  1.150250  0.024252  1.278846 -1.897882 -1.805178   
8 -0.553925 -0.496169  1.150250  0.024252  1.278846 -1.897882 -1.805178   
9 -0.553925 -0.496169  1.150250  0.024252  1.278846 -1.897882 -1.805178   

         7         8         9         10        11        12        13  \
0  1.010520  0.247055 -0.289277  3.179021  0.701933  2.375838 -0.246993   
1  0.340487  0.271904  0

In [ ]:
print(node_features)

NameError: name 'node_features' is not defined